# Mapping Yelp Search Results - Part 1

- 04/26/22


## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [2]:
# Load API Credentials

with open('/Users/vinosraj/.secret/yelp_api.json') as f: #default 'r' default so dont need to add
    login = json.load(f)
login.keys()


dict_keys(['client-id', 'api-key'])

In [3]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

### Define Search Terms and File Paths

In [4]:
# set our API call parameters and filename before the first call


LOCATION = 'Rochester, MN'
TERM = 'Asian'

In [6]:
## Specify fodler for saving data
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)


In [7]:
LOCATION.split(',')[0]

'Rochester'

In [8]:

# Specifying JSON_FILE filename (can include a folder)

JSON_FILE = f"{FOLDER}{TERM}_{LOCATION.split(',')[0]}.json"
JSON_FILE

'Data/Asian_Rochester.json'

### Check if Json File exists and Create it if it doesn't

In [11]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)

## If it does not exist: 
if file_exists == False:

    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)

     ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/Asian_Rochester.json not found. Saving empty list to file.


### Load JSON FIle and account for previous results

In [12]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')


- 0 previous results found.


### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [13]:
# use our yelp_api variable's search_query method to perform our API call
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                             )
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [14]:
## How many results total?
total_results = results['total']
total_results

76

- Where is the actual data we want to save?

In [15]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [17]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil(results['total']/ results_per_page)
n_pages

4

In [21]:
for i in tqdm_notebook( range(0,n_pages+1)):
    ## The block of code we want to TRY to run
    try:
        time.sleep(.2)   
        
        ## Read in results in progress file and check the length
        ## Load previous resuls and use the len of results as offset
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
        ## save number of results for to use as offset
        n_results = len(previous_results)
        ## use n_results as the OFFSET 
        results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results+1)
    
        ## append new results and save to file
        previous_results.extend(results['businesses'])
    
        #display(previous_results)
        with open(JSON_FILE,'w') as f:
            json.dump(previous_results,f)
    ## What to do if we get an error/exception.
    except Expection as e:
        print(e)
        break


  0%|          | 0/5 [00:00<?, ?it/s]

## Open the Final JSON File with Pandas

In [23]:
df = pd.read_json(JSON_FILE)
display(df.head(), df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,MLjDJR4N-h_6_LBJwnHyLQ,ichi-tokyo-rochester,Ichi Tokyo,https://s3-media4.fl.yelpcdn.com/bphoto/jVMnFE...,False,https://www.yelp.com/biz/ichi-tokyo-rochester?...,168,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 43.9923896789551, 'longitude': -9...","[delivery, pickup]",$$,"{'address1': '102 20th St SE', 'address2': 'St...",+15072882577,(507) 288-2577,3048.998621
1,q-nxw7sQSdaqQmZcCuiMfQ,pho-chau-rochester,Pho Chau,https://s3-media3.fl.yelpcdn.com/bphoto/uKn8Tu...,False,https://www.yelp.com/biz/pho-chau-rochester?ad...,374,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.0,"{'latitude': 44.033622, 'longitude': -92.462526}","[delivery, pickup]",$,"{'address1': '1014 Broadway Ave N', 'address2'...",+15072813342,(507) 281-3342,2262.723330
2,JlowEYIonJ-rWLLqQdm_aQ,asia-fusion-rochester-2,Asia Fusion,https://s3-media3.fl.yelpcdn.com/bphoto/Wm86-o...,False,https://www.yelp.com/biz/asia-fusion-rochester...,78,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.0,"{'latitude': 44.052304, 'longitude': -92.521926}","[delivery, pickup]",$,"{'address1': '2711 Commerce Dr NW', 'address2'...",+15072587992,(507) 258-7992,5227.646429
3,egqrH1I4WGdCZaWMUcWj5Q,thai-pop-rochester,Thai Pop,https://s3-media2.fl.yelpcdn.com/bphoto/7AU9IN...,False,https://www.yelp.com/biz/thai-pop-rochester?ad...,41,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.5,"{'latitude': 44.02030690857195, 'longitude': -...",[],NaN,"{'address1': '4 3rd St SW', 'address2': '', 'a...",+15079442025,(507) 944-2025,1288.933944
4,Oq9fbffSimDnpO3JyiSYvQ,wabi-sabi-rochester,Wabi Sabi,https://s3-media2.fl.yelpcdn.com/bphoto/kMhifd...,False,https://www.yelp.com/biz/wabi-sabi-rochester?a...,7,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 44.010489, 'longitude': -92.441017}",[delivery],NaN,"{'address1': '1505 12th St SE', 'address2': ''...",+15072581058,(507) 258-1058,3105.412693


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
63,RkgxuYCs_kpRZPMKO9BPCA,cb3-burgers-and-brews-rochester,CB3 Burgers & Brews,https://s3-media2.fl.yelpcdn.com/bphoto/P1iwbN...,False,https://www.yelp.com/biz/cb3-burgers-and-brews...,71,"[{'alias': 'newamerican', 'title': 'American (...",2.5,"{'latitude': 44.0235141, 'longitude': -92.4682...",[],$$,"{'address1': 'W Center & 4th Ave NW', 'address...",+15073611910,(507) 361-1910,1114.584134
64,8cYa3RGHozMVCGjr19Tjyw,hy-vee-rochester-3,Hy-Vee,https://s3-media2.fl.yelpcdn.com/bphoto/5XjmFE...,False,https://www.yelp.com/biz/hy-vee-rochester-3?ad...,34,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.5,"{'latitude': 44.06037, 'longitude': -92.46938}","[delivery, pickup]",$$,"{'address1': '500 37th St NW', 'address2': '',...",+15072891815,(507) 289-1815,4816.470627
65,Fi0gIph_MEgpqnyEliRg8Q,sport-clips-haircuts-of-rochester-windsor-reta...,Sport Clips Haircuts of Rochester Windsor Reta...,https://s3-media3.fl.yelpcdn.com/bphoto/cYpPEV...,False,https://www.yelp.com/biz/sport-clips-haircuts-...,6,"[{'alias': 'menshair', 'title': 'Men's Hair Sa...",2.5,"{'latitude': 44.0057248677443, 'longitude': -9...",[],$$$,"{'address1': '414 Crossroads Dr', 'address2': ...",+15072813798,(507) 281-3798,1486.461072
66,sLaEWZ-O_kv7yDfZgkTYVg,panera-bread-rochester-2,Panera Bread,https://s3-media3.fl.yelpcdn.com/bphoto/Tf2LDV...,False,https://www.yelp.com/biz/panera-bread-rocheste...,27,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.5,"{'latitude': 44.0619979738136, 'longitude': -9...","[delivery, pickup]",$,"{'address1': '3780 Market Place Drive NW', 'ad...",+15072809185,(507) 280-9185,5282.627356
67,t3UmDYtw38D9-NkO00j5eA,buffalo-wild-wings-rochester-2,Buffalo Wild Wings,https://s3-media1.fl.yelpcdn.com/bphoto/p6NpM1...,False,https://www.yelp.com/biz/buffalo-wild-wings-ro...,47,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",3.0,"{'latitude': 44.0773649, 'longitude': -92.5159...",[],$$,"{'address1': '3458 55th St NW', 'address2': ''...",+15075360717,(507) 536-0717,7344.422481


In [24]:
df.duplicated(subset='id').sum()

0

In [29]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Asian_Rochester.csv.gz'

In [31]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression='gzip',
                index=False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [32]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))##is csv_file =JSON_FILE.replace('.json','.csv.gz')

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 66,128 Bytes
CSV.GZ FILE: 9,968 Bytes
the csv.gz is 6.634028892455858 times smaller!


## Next Class: Processing the Results and Mapping 